# EMPIRICAL MACROECONOMICS
*Francesco Franco, Nova SBE*
## Class 1 Python warm-up on Sims lecture
Words in bold are commands to be typed in the Anaconda prompt.
### Setting up the class environment

1. Install [Anaconda](https://www.anaconda.com/). 
   - To check the conda version: **conda list anaconda$**
2. Create you work environment to be able to reproduce your work
   - to check the environments you have: **conda env list**
   - to create the Class environment: **conda create -n ClassEM python=3.6 notebook numpy pandas matplotlib xlrd statsmodels pandas-datareader seaborn**
   - to activate the environment: **conda activate ClassEM**
   - to deactivate the environment: **conda deactivate**
   - to list packages in your environment: **conda list -n ClassEM**
   - to export your packages in a list: **conda list --export > package-list.txt**
   - to create an environment with your packages: **conda create -n myenv --file package-list.txt**   


In [ ]:
'''
We always start the code by importing the required package into memory
''' 
import numpy as np                                       #import package for data array manipulation
import pandas as pd                                      #import package for data analysis         
from scipy.optimize import minimize                      #import package for scientific computing
import matplotlib.pyplot as plt                          #import package for plotting
from pandas.util.testing import assert_frame_equal
from pandas_datareader.data import DataReader            # DataReader allows to download data from the main online
                                                         # repositories : Eurostat, FRED, OECD, ..
import seaborn as sns
from scipy.stats import multivariate_normal,norm, invgamma, uniform 

### Package Pandas: forget EXCEL
Help for common packages is accessible through the help tab of the Jupyter notebook

This section loads the data and present basic data manipulation commands

#### Loading and exploring data

In [ ]:
'''
PANDAS_DATAREADER, simplest tool. we will see more advanced ways to download data using SDMX
'''
start = '1929' #start date                                             
end = '1943' #end date
data = DataReader(['PCECCA','GPDICA','GCECA'], 'fred', start=start, end=end) #Data from FRED, you need to find the code in the webiste



In [ ]:
data.info() # the info method gives you a lot of information 

In [ ]:
data.head() # check the first 5 rows with method .head(). 

Let us create output
$$Y_t = C_t + I_t + G_t$$

In [ ]:
haavelmo_data = data.copy() # create a new copy
haavelmo_data.columns = ['c', 'i', 'g'] # rename columns
haavelmo_data['y'] = haavelmo_data['c'] + haavelmo_data['g'] + haavelmo_data['i'] # create a new column with output

In [ ]:
'''
of course you can read from files, csv are prefererd but almost any format is readable
you need xlrd module for excel files 
'''
df = pd.read_excel('../Data/Sims.xls')


In [ ]:
df.info() # Check the data with the method .info()

In [ ]:
df.describe().T # Summary Statistics

In [ ]:
df.head() # check the first 5 rows with method .head(). NOTICE indexing starts with 0 !

In [ ]:
'''
Selecting a column or a row : there are many ways to select a comun let us converge on one
let us reduce the data set to the first 5 rows
'''
df5 = df.head().copy()

In [ ]:
df5['PCECCA'] # select a column with the name

In [ ]:
'''
Purely integer-location based indexing for selection by position .iloc 
nice but you need to know the location
'''
df5.iloc[1:3,0:4] # notice it starts from index 1 (row 2) up to index 2 but does not include index 3 same for columns

In [ ]:
'''
Purely label-location based indexer for selection by label .loc
can build very complex queries in your table
'''
sel = (df5['date']>='1930-01-01') & (df5['date']<='1932-01-01') # logical operator &
df5.loc[sel,['date','GPDICA','Y']]

#### Operators
we have used a logical operator in the above selection
- $|$ :or
- & :and
- $>$ :gt
- $<$ :lt
- $>=$ :ge
- $<=$ :le
- $==$ :eq
- $!=$ :ne

In [ ]:
df.columns # print the name of the columns

In [ ]:
df.rename(columns={'Y': 'GDP'},inplace=True) # rename a column, attention to inplace

In [ ]:
df.columns # print the name of the columns

#### Built-in function

In [ ]:
df['GDP'].sum() # as a method

In [ ]:
sum(df['GDP']) # as a function (procedure)

#### Performance

In [ ]:
%timeit -n 100 df['GDP'].sum()

In [ ]:
%timeit -n 100 sum(df['GDP'])

### Package Numpy: numerical operations


In [ ]:
'''
if speed is really an issue, namely BIG dataset, or loops, then transform dataframe into numpy arrays
'''
GDP = np.array(df['GDP'])
%timeit -n 100 np.sum(GDP)

### Package Matplotlib: plot everything


In [ ]:
'''
plot Haavelmo data
'''
plt.figure(figsize=(12,6)) # declare a figure
plt.plot(haavelmo_data) # plot the data
plt.xlabel('Date') # add labels and title
plt.ylabel('Billion Chained Dollar')
plt.title('Haavelmo data')
plt.legend(haavelmo_data.columns) # add legend
plt.grid() # add a grid
plt.show()

In [ ]:
haavelmo_data.plot(grid=True,figsize=(12,6)) # Note that Panda as a plot method, slighlty less control but very practical
plt.show()

In [ ]:
'''
Subplots
'''
plt.figure(figsize=(16,10))
plt.subplot(221)
plt.plot(haavelmo_data.c)
plt.xlabel('Date')
plt.ylabel('Billion Chained Dollar')
plt.title('Consumption')
plt.grid()
plt.subplot(222)
plt.plot(haavelmo_data.i)
plt.xlabel('Date')
plt.ylabel('Billion Chained Dollar')
plt.title('Investment')
plt.grid()
plt.subplot(223)
plt.plot(haavelmo_data.g)
plt.xlabel('Date')
plt.ylabel('Billion Chained Dollar')
plt.title('Government expenditure')
plt.grid()
plt.subplot(224)
plt.plot(haavelmo_data.y)
plt.xlabel('Date')
plt.ylabel('Billion Chained Dollar')
plt.title('Output')
plt.grid()
plt.show()


In [ ]:
haavelmo_data.plot(subplots=True,layout=(2,2),figsize=(16,10),grid=True)
plt.show()

### Haavelmo model

> Haavelmo Statistical Testing of Business-Cycle Theories
The modified Haavelmo Model (Sims Nobel Lecture) is:


$$C_t = \beta + {\alpha}Y_t+\epsilon_t$$

$$I_t = \theta_0 + \theta_1(C_t - C_{t-1}) + \eta_t$$

$$Y_t = C_t + I_t + G_t$$

$$G_t = \gamma_0 + \gamma_1G_{t-1} + v_t$$

Where $\epsilon_t$,$\eta_t$, and $v_t$ are Normally distributed disturbance with constant variance and zero mean.  Substituting for $Y_t$ the system is therefore


\begin{equation}
C_{t}\left(1-\alpha\right)-{\alpha}I_t -{\alpha}G_t = \beta + \epsilon_t
\end{equation}

\begin{equation}
-\theta C_{t}+I_{t}=\theta_0 -\theta C_{t-1}+\eta_{t}
\end{equation}

\begin{equation}
G_t = \gamma_0 + \gamma_1G_{t-1} + v_t
\end{equation}

or in matrix terms

$${\Gamma_0}X_t = C + {\Gamma_1}X_{t-1} + U_t$$
where
$$\Gamma_0=\begin{pmatrix}
1-\alpha&-\alpha&-\alpha\\
-\theta&1&0\\
0&0&1
\end{pmatrix}$$

$$C=\begin{pmatrix}
\beta\\
\theta_0\\
\gamma_0
\end{pmatrix}$$


The reduced form is
$$X_{t}=\Gamma_{0}^{-1}C+\Gamma_{0}^{-1}\Gamma_{1}X_{t-1}+\Gamma_{0}^{-1}U_{t}$$

$$X_{t}=A+BX_{t-1}+V_{t}$$


### Code for the Model
- Python is an “object-oriented programming language.” This means that part of the code is implemented using a special construct called classes.
- Python also allow a "a functional approach" which is the one we will follow in the course.
- Programmers use classes to keep related things together. This is done using the keyword “class,” which is a grouping of object-oriented constructs.
- A class is a code template for creating objects. Objects have member variables and have behaviour associated with them. In python a class is created by the keyword class. 
- A class by itself is of no use unless there is some functionality associated with it. Functionalities are defined by setting attributes, which act as containers for data and functions related to those attributes. Those functions are called methods.
- You can also provide the values for the attributes at runtime. This is done by defining the attributes inside the init method.

#### example of a class

In [ ]:
# We define Haavelmo as class (base)
# The __init__ function is a special method that is run whenever an
# object is created. The self parameter is a reference to the current instance of the class.
class Haavelmo():
    
    def __init__(self,params=None):
               
        # Initialize parameters
        if params is not None:
            self.update(params)
            
    def update(self, params):
        
        # update the parameter values during estimation  
        self.alpha   = params[0]
        self.beta    = params[1]
        self.theta1  = params[2]
        self.theta0  = params[3]
        self.gamma1  = params[4]
        self.gamma0  = params[5]
        self.eps_std = params[6]
        self.eta_std = params[7]
        self.v_std   = params[8]    
        
    def Gamma_0(self):
        Gamma0 = np.array([
                          [1-self.alpha, -self.alpha, -self.alpha],
                          [-self.theta1, 1, 0],
                          [0, 0, 1],
                          ])    
        return Gamma0
    
    def C_(self):
        C = np.array([
                     [self.beta],
                     [self.theta0],
                     [self.gamma0],
                     ])        
        return C

    def Gamma_1(self):
        Gamma1 = np.array([
                          [0, 0, 0],
                          [-self.theta1, 0, 0],
                          [0, 0, self.gamma1],
                        ])       
        return Gamma1
    
    def VV_(self):
        VV = np.array([
                      [self.eps_std**2, 0, 0],
                      [0, self.eta_std**2, 0],
                      [0, 0, self.v_std**2],
                      ])
        return VV

#### example of a function
The likelihood is

$$f_{X^{T}}(X^{T};\psi)=f_{X_{0}}(X_{0};\psi)\prod_{t=1}^{T}f_{X_{t}|X_{t-1}}\left(X_{t}|X_{t-1};\psi\right)$$

Then the negative loglikelihood of our model is

$$\mathcal{L}(\psi)=-\left(Tn/2\right)+\left(T/2\right)log\left|\Gamma_{0}\right|^{2}-\left(T/2\right)log\left|D\right|-0.5\sum_{t=1}^{T}\left[\left(\Gamma_{0}X_{t}-C-\Gamma_{1}X_{t-1}\right)'D^{-1}\left(\Gamma_{0}X_{t}-C-\Gamma_{1}X_{t-1}\right)\right]$$

In [ ]:
def neg_loglike(theta,endog,exog):
    
    '''Compute the negative of the likelihood of Haavelmo model
       
       Input
       theta:the initial guesses for the parameters
       endog:the endogenous variables
       exog:the exogenous variables (here the lagged endogenous variables)
       
       Ouput
       penalized negative loglikelihood
       
    '''
    params = theta
    mod = Haavelmo(params) # use the Haavelmo class
    Xhat = mod.Gamma_0()@endog
    U = Xhat - mod.C_() - mod.Gamma_1()@exog
    T = U.shape[1]
    n = U.shape[0]
    temp = np.zeros(T-1)    
    for t in range(0,T-1):
        temp[t] = U[:,t].transpose()@np.linalg.inv(mod.VV_())@U[:,t]        
    ll = ( -T*n/2*np.log(2*np.pi) + T/2*np.log(np.linalg.det(mod.Gamma_0()))
           -0.5*temp.sum() - T/2*np.log(np.linalg.det(mod.VV_())) )                 
    ll = ( ll - 1000*min(0,1 -params[0]*(1+params[2]))**2 
              - 1000*min(0,params[2])**2
              - 1000*min(0,params[6])**2
              - 1000*min(0,params[7])**2
              - 1000*min(0,params[8])**2 )   
    return -ll

## Data preparation

In [ ]:
'''
Prepare the data X and X(-1) this would be the Haavelmo original sample
'''
endog = np.array(haavelmo_data[['c','i','g']]).T  
endog = endog[:,1:] # drop first observation because of one lag

exog = np.array(haavelmo_data[['c','i','g']].shift(1)).T      
exog = exog[:,1:]   # drop first observation because of one lag

In [ ]:
'''
Prepare the data DlogX and DlogX(-1)
We will use the growth rate to be consistent with stationarity
'''

haavelmo_data['lc'] = np.log(haavelmo_data['c'])
haavelmo_data['li'] = np.log(haavelmo_data['i'])
haavelmo_data['lg'] = np.log(haavelmo_data['g'])

haavelmo_data['dlc'] = haavelmo_data['lc'] - haavelmo_data['lc'].shift(1)
haavelmo_data['dli'] = haavelmo_data['li'] - haavelmo_data['li'].shift(1)
haavelmo_data['dlg'] = haavelmo_data['lg'] - haavelmo_data['lg'].shift(1)

endog = np.array(haavelmo_data[['dlc','dli','dlg']]).T       
endog = endog[:,2:] # drop first 2 observations because of one lag

exog  = np.array(haavelmo_data[['dlc','dli','dlg']].shift(1)).T       
exog  = exog[:,2:]   # drop first 2 observations because of one lag

In [ ]:
# Plot data in growth rate
plt.figure(figsize=(16,10))
plt.subplot(221)
plt.plot(haavelmo_data.dlc)
plt.xlabel('Date')
plt.ylabel('Billion Chained Dollar')
plt.title('Consumption')
plt.grid()
plt.subplot(222)
plt.plot(haavelmo_data.dli)
plt.xlabel('Date')
plt.ylabel('Billion Chained Dollar')
plt.title('Investment')
plt.grid()
plt.subplot(223)
plt.plot(haavelmo_data.dlg)
plt.xlabel('Date')
plt.ylabel('Billion Chained Dollar')
plt.title('Government expenditure')
plt.grid()
plt.show()

### Minimization using Scipy package routines

In [ ]:
'''Minimization of -LogLikelihood, constrained
'''
# initial guesses        alpha  beta  theta1 theta0 gamma1 gamma0 eps_std eta_std v_std        
theta_start = np.array([ 0.05, 0.5,  0.05,   0.5,  0.991,   0.2,     0.1,   .1,     .1])
# Nelder-Mead, BFGS, 
res = minimize(neg_loglike, theta_start,args=(endog,exog),method="BFGS",
               options={'disp': True,'maxiter':100})

In [ ]:
'''
Estimated parameters
'''
sol = pd.DataFrame(res.x,index=[r'$\alpha$',r'$\beta$', r'$\theta_1$',
                                r'$\theta_0$',r'$\gamma_1$',r'$\gamma_0$',
                                r'$\sigma_{\epsilon}$',r'$\sigma_{\eta}$',r'$\sigma_{\nu}$'],columns=['parameters'])
sol['SE']=np.sqrt(np.diagonal(res.hess_inv))
sol['T-stat']=sol['parameters']/sol['SE']
sol

Reminder to interpret the parameters

$$C_t = \beta + {\alpha}Y_t+\epsilon_t$$

$$I_t = \theta_0 + \theta_1(C_t - C_{t-1}) + \eta_t$$

$$Y_t = C_t + I_t + G_t$$

$$G_t = \gamma_0 + \gamma_1G_{t-1} + v_t$$

## Bayesian analysis

A nice and common way to highlight the differences between frequentist and bayesian analysis is to compare the confidence interval of frequentists with the corresponding notion in Bayesian statistics, credible interval.

- Confidence interval is from the frequentist's approach, where the parameter is fixed. The confidence interval is based on the repetition of the observations. A 95% confidence interval means that repeating the experiment to measure the parameter a large number of times and calculating the interval for each experiment, 95% of the intervals will contain the value of the parameter. This goes back to the fact that the data is random.


- Credible (or probability) interval stems from probabilities, that is, the Bayesian approach. This means that the parameter is random and we can say that, given the data, there is a 95% chance that the true value of the parameter is in the interval.

Markov chain Monte Carlo (MCMC) methods
In Bayesian statistics the parameters are not constant but random variables with a distribution. We are now looking for

$$\pi(\psi|X^T)= \frac{f_{X^T}(X^T|\psi)\pi(\psi)}{f_{X^T}(X^T)}$$

where the first term of the denominator on the RHS is the likelihood and the second the prior distribution of the parameters.The LHS is the posterior distribution and is the quantity of interest. MCMC methods allow to sample from ths posterior distribution.

One of the most simple algorithm to implement a MCMC is the Metropolis-Hastings algorithm. The idea is to construct a Markov chain for $\psi$

1. Given the current value of $\psi_{s-1}$, propose a new value $\psi^*$ selected from a proposal $q(\psi;\psi_{s-1})$

2. With probability $\alpha(\psi_{s-1},\psi^*)$ the proposed value is accepted if not the chain remains in place
now $$\alpha(\psi_{s-1},\psi^*) = min\Big({\frac{\pi(\psi^*|X^T)q(\psi^*;\psi_{s-1})}{\pi(\psi_{s-1}|X^T)q(\psi_{s-1};\psi^*)}},1\Big)$$

 $$  \alpha(\psi_{s-1},\psi^*)                                    = min\Big({\frac{f_{X^T}(X^T|\psi^*)\pi(\psi^*)q(\psi^*;\psi_{s-1})}{f_{X^T}(X_T|\psi_{s-1})\pi(\psi_{s-1})q(\psi_{s-1};\psi_{s-1}^*)}},1\Big)$$
 
 and using a proposal distribution that satistifes $q(\psi_{s-1};\psi_{s-1}^*)=q(\psi^*;\psi_{s-1})$. A convienent choice is
 
 $$\psi^*=\psi_{s-1} +\epsilon_s, \epsilon_s \sim{N(0,\Sigma_\epsilon)}$$

In [ ]:
prior_alpha = uniform(loc =0, scale=3)   # Specify priors
prior_beta = uniform(loc =0, scale=3)        
prior_theta = uniform(loc =0, scale=3)         
prior_theta0 = uniform(loc =0, scale=3)         
prior_gamma1 = uniform(loc =0, scale=2)         
prior_gamma0 = uniform(loc =0, scale=2) 
prior_eps_std = invgamma(5)
prior_eta_std = invgamma(5)
prior_v_std  = invgamma(5)

rw_proposal = multivariate_normal(cov=res.hess_inv*0.4)   # Specify the random walk proposal
                                                          # an important aspect: the initial covariance matrix is
                                                          # the inverse of the Hessian of likelihod and the parameter
                                                          # is set as to obtain a 25% acceptance rate


In [ ]:
'''
Metropolis Hastings algorithm
'''

n_iterations = 1000
trace = np.zeros((n_iterations + 1, 9)) # Create storage arrays for the traces
trace_accepts = np.zeros(n_iterations)
trace[0] = [0.05, 0.5, 1, 0.5, 0.991, 0,0.1,.1,.1] # Initial values
# Iterations
for s in range(1, n_iterations + 1):
    proposed = trace[s-1] + rw_proposal.rvs() # step 1
    acceptance_probability = np.exp(-neg_loglike(proposed,endog,exog) + neg_loglike(trace[s-1],endog,exog) +
    prior_alpha.logpdf(proposed[0]) +
    prior_beta.logpdf(proposed[1]) + prior_theta.logpdf(proposed[2]) +
    prior_theta0.logpdf(proposed[3])+ prior_gamma1.logpdf(proposed[4]) +
    prior_gamma0.logpdf(proposed[5])+ prior_eps_std.logpdf(proposed[6]) + 
    prior_eps_std.logpdf(proposed[7]) + prior_eps_std.logpdf(proposed[8]) -
    prior_alpha.logpdf(trace[s-1, 0]) -
    prior_beta.logpdf(trace[s-1, 1]) - prior_theta.logpdf(trace[s-1, 2]) -
    prior_theta0.logpdf(trace[s-1, 3])- prior_gamma1.logpdf(trace[s-1, 4]) -
    prior_gamma0.logpdf(trace[s-1, 5])- prior_eps_std.logpdf(trace[s-1, 6]) - 
    prior_eps_std.logpdf(trace[s-1, 7]) - prior_eps_std.logpdf(trace[s-1, 8])) # compute odd ratio
    pp = uniform.rvs()                                
    if acceptance_probability > pp: # step 2
        trace[s] = proposed
        trace_accepts[s-1] = 1
    else:
        trace[s] = trace[s-1]

In [ ]:
plt.plot(np.cumsum(trace_accepts)/np.arange(1, len(trace_accepts)+1)) # Plot the acceptance rate

In [ ]:
plt.plot(trace[:,4]) # plot the value sample gamma1 from the posterior

In [ ]:
plt.hist(trace[:,4]) # Plot the histogram of gamma1

In [ ]:
index=['alpha','beta', 'theta1','theta0','gamma1','gamma0','eps_std','eta_std','v_std']
fig, axes = plt.subplots(9, 2, sharex='col')
fig.set_size_inches(12, 40)
for i in range(9):
  axes[i][0].plot(trace[:,i])
  axes[i][0].title.set_text(str(index[i]))
  sns.kdeplot(trace[:,i], ax=axes[i][1], shade=True)
  axes[i][1].title.set_text(str(index[i]))
axes[9 - 1][0].set_xlabel("Iteration")
axes[9 - 1][1].set_xlabel("coefficient")
fig.tight_layout()
plt.show()